In [4]:
from binascii import hexlify

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.serialization import (
    Encoding,
    PrivateFormat,
    NoEncryption,
)
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

In [5]:
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding

### Generate DH paramaters

In [6]:
print("Generating parameters...")
parameters = dh.generate_parameters(generator=2, key_size=2048)
print("\nModule:\n", parameters.parameter_numbers().p)
print("\nGen:", parameters.parameter_numbers().g)

Generating parameters...

Module:
 22010812045490892213697667632013057206201714936671414017966118477150296253225714736423713520127741929002772500501078475263107949271293637583019120985954939233533691784773713093969853942735666666575538200852595979649957952363089067153623690460466464386608934877678696849927863913848365452588019116239998694785552642669765225187647324986653400375365712939621957463402598956927454858079704559220393063379030775235733900756802704309348910390830358851328080707123329540745425099801154119447470875911903372928633192213671173809242540529577968845371750685781918999002441065382303360242031969842005087814664227857606015993319

Gen: 2


# Alice side

### Generate DH public/private keys

In [42]:
alice_private_key = parameters.generate_private_key()  # a
alice_public_key = alice_private_key.public_key()  # g^a
alice_public_key_bytes = alice_public_key.public_numbers().y.to_bytes(2048 // 8, 'big')

print(f"Key size: {alice_public_key.key_size}")
print(f"Key type: {type(alice_public_key.public_numbers().y)}")
print(f"Value: {alice_public_key_bytes}")

Key size: 2048
Key type: <class 'int'>
Value: b'A^\x92$\'\x92\x015\xe5\x89-f%\xa2\x0f\x146\x90\xf2)\x88\x1c\x95\xf1\x99\xcd\x89\x1e\x19\xc1m\x1c\xae\xa9\x9b-\x8b\x9e\xa0\xb3H}$\x88\x8cv\xfb$r\x80\xb6\xc1\x07\x1a\x92\xa0\x13\xc8\xf9\x05\n"d\xbeM\x8b\xac=\xe1r\xbb\x02a\xd5\xef\x90\xed\xd2z\xdd\xcd\x8a\xff7\x85\xf3N\x1b,Ss\xfce?Y\xc5\xdf\x00\xcf\r\xb5\xd3\xf0\xd4Yp\xe3\xc4\x9e\xe2\xf6e\xf3T>\xd0\x1c\xb5\xd3\xde\x03\xea9\xcc\x9a\xac6A\x11\xc6\xfb\xd2\xae\xe5K[\xa0I\x07\xff}\x80\xc9\x94\x1b\xd7\x9e\x9c\xb8\xb01\xde\xf2\xa1ng\x191\x8e\xe6\xd0\x92\x1d:CYN\x10\x98TN\xc9\xa5\xa5+\xff\xe0\xc2\x8e\xe4\x087cv\x96\x1b\x90\xb02\tz\xf0\x86*\xb7\xa4aQ\xea`\xb6d\x8f_\xd8\xd5\x8d\xeb\xde\x8d\x92\xa61\tP\xd8#\xfcmoF\x0cxw\x035\\\x89\xbb\xa0\'\x96\xd7y\x8d\xcf\x83v\xf7.q\xea\x95D0\x0e\xc1\x98\xc1\x89\xc4\xaard\xd0\x00'


### Generate RSA public/private pair

In [43]:
alice_rsa_private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)
alice_rsa_public_key = alice_rsa_private_key.public_key()
alice_rsa_public_key_pem = alice_rsa_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo,
)
print(alice_rsa_public_key_pem)

b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAtqQxR6DVxWDIyRc5pK8n\ngg8F8dGL/w+RnlMsXADBEvv9mjK7m9Yo0prgTqcDQeJ6jTX7Stvq47jJQ0fC362j\nYKk8oe5gpFUKfsYkJuxtlPYJX0Gp0NJZds9o/CDsf6roHhCgSlV7M5W6swdEZ5gV\n1fzCYBoMM0Xk229DQxKcE3C+jTVcUv+c7bBB6gPCOu/UWwW7XCBIFvWXualTYFDC\nQFsYxeQAjD0VO72b9p/6FYUF/dki0IjcJMUIuPrWqqD9hRhqHAJxE8B4d+tjNXJV\na6upcdY/z8sVRFbFeGG1AO/Nd11NoS3cfjLDUD37YYU6taZisxwcryRRmiG4tKhm\n3QIDAQAB\n-----END PUBLIC KEY-----\n'


#### Share Alice's RSA public key

In [44]:
# alice_rsa_public_key --> all


### Generate RSA signature for Alice's DH key

In [45]:
alice_signature = alice_rsa_private_key.sign(
    alice_public_key_bytes,
    padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
    hashes.SHA256(),
)

In [46]:
hexlify(alice_signature)

b'7ffb05c569c6511caa3396487f6fd6b9dfea1acec3134eef55145fcc277e3dae86d14aa479e95942df6bea79ec353d3c7a27b2e56907da75c854b550346e9278b18738c512381b41c00d8026a834e028ddbf25a5bc927027617ea9a69141b94b98501c06e2e5a60a1b24a5ddc3a406bd784ba65d4788754719c1902df02efdc4e52d127371431e0ed3f754a056f6cd44a1fbb833e6243793a66ca34ea3c4004dc3cf80e8e13e3536327e479bc9831d1c5a269f095b514da5445df2ec224b4321e89194233add149ad8a5328e76adbaa31302168a138ea3c0f26e99dbe94525ebba6de868cca03493154cb497d558f3d9118281dae9cce4d90001cf650670d251'

# Bob side

### Generate DH public/private keys

In [47]:
bob_private_key = parameters.generate_private_key()  # b
bob_public_key = bob_private_key.public_key()  # g^b
bob_public_key_bytes = bob_public_key.public_numbers().y.to_bytes(2048 // 8, 'big')

print(f"Key size: {bob_public_key.key_size}")
print(f"Key type: {type(bob_public_key.public_numbers().y)}")
print(f"Value: {bob_public_key_bytes}")

Key size: 2048
Key type: <class 'int'>
Value: b'C\x96\xd6\x90\xe1\xee\xba\xde:6dT.\x9a\x8ex\x19.Cl{\x1d\xda\x18\xbd\xbf\x1a\xa2\x8f\x8a$[\x06\xeb\xa1\xdc\x11B\x11\x9b\xe4\x90\xe6\xf4\xbe\xd8Z\x93a\xbc\xac0\xc2\xdc\xdbMH\xf6c\xddA\xbb~\x85\xe2/\xba\x1eD\x93\'\x13\x0b\xce}(\xc3\xf1MG\xfc"\xed\xf8\xfe\x8f\x1f[\xe5\xf6\xc3\x98\xa8Fg\xc2\xd3J&\x9a^\x95zC\x11\x1c\xf9\xe4nm\x960B\xc1l-B\x9b\xa4\xe3\x82z\xd7\x1e\xe5\xd3.\xd6{\xc3h\xf8Y\xcaw\x9bx_L]\x8dK&V\xa2\xc8#\x9a\xb3\xfc?\xcd\xa2\xa4S\xbb<\xe1G"@\x04v\x9b\xb7U\xe3F_\x82-\x7fHi\xe7\x02\x92\xe6\xb2\xe3e\xf2\xeb\xe4l\x0b\x17\xe1\x1f\xd7\xa6\xe1\x08\xa6u\x97\x1aqU\x014\x06\xd6\xb7\x1c3\xe6\xb2\xcbZ@\x04|\x9a\x16f\xff\xe3\xda\x9d9\'5?E\xab\x85\xef\x1bs\xb3\\B\x7f\xa4\xe5\xc1S\x822;\xbf\xce\x1d\xc9\xa3LOf\xf3Y[\x90\xb8\x12\x96'


### Generate RSA public/private pair

In [48]:
bob_rsa_private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)
bob_rsa_public_key = bob_rsa_private_key.public_key()

#### Share Bob's RSA public key

In [49]:
# bob_rsa_public_key --> all

In [50]:
bob_rsa_public_key_pem = bob_rsa_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo,
)
print(bob_rsa_public_key_pem)

b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAvwfG4ZIrk2uj74a+GchH\nf3WBPESQosjYwCkOzxlrPXvnMWEbfE8LraoMeXmEBLkUQN6MPMZeBX8d4V2ViKdy\nn3hYY4ohMp2uf3j2AC54T72QIPooEPIGgiJxhcwJyvuGmqVtcUjXI7MQtNTFBprP\nSzfeVJCYeVbA2PS/u4zufTinrSkB5KEwgCVZ7bFYGNOsFama0gphAF4jqivrzPDE\nf7U1Frz3+EsTV5P2UPp9edmGUlU4GYugsqyizPi3G9AbMIpMwMNCO8UJnVXDtxu0\nvz3hoWwnEoESQjhWv1Z87STLRibU5pfR+E8q9654Uv02LjAQ7F5f66pfVsZtjGJy\nLwIDAQAB\n-----END PUBLIC KEY-----\n'


### Generate RSA signature for Bob's DH key

In [51]:
bob_signature = bob_rsa_private_key.sign(
    bob_public_key_bytes,
    padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
    hashes.SHA256(),
)

hexlify(bob_signature)

b'6e8b96456816e30577123de8a96b4f6c5e5f186266df4f01df9a5b9feb7bcffa5acb583352063a6a65a4847157be11afe7715c3081ccafd2daf43c466f1ff4655633a2d262bd0a5017c7e7efbbd99621c308d6dc9add1712adab84dcc8d48a70d5a4ba8588881413420e02be4dcca4c8f37baff2581e609fdc9a9efb6636959c2066c6c33da75fdab3f0518088922c41b68be5f0cca3ee7d279413958759f25fce0b1705403c6ea5158d16d805fe310be73060cf5c4d0af28922002bb80dc9491558196f2ba6aa8b379a1970c6bf595aad026f848fcdc21f774f03fbd7689d3eacb5fe122a6b0e2be7f736a52dbbdd7e990c2c9492f736c909b9b7848b44438b'

### Keys exchange

In [5]:
# Alice --> Bob:    alice_public_key + signature
# Bob --> Alice:    bob_public_key + signature

# Check signatures of keys

#### Verify Alice DH public key vs signature

In [53]:
res = alice_rsa_public_key.verify(
    alice_signature,
    alice_public_key_bytes,
    padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
    hashes.SHA256(),
) is None
print(f"Alice DH key verification is passed: {res}")

Alice DH key verification is passed: True


#### Verify Bob DH public key vs signature

In [57]:
res = bob_rsa_public_key.verify(
    bob_signature,
    bob_public_key_bytes,
    padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
    hashes.SHA256(),
) is None
print(f"Bob DH key verification is passed: {res}")

Bob DH key verification is passed: True


### Generate shared simmetric key

In [6]:
# Alice
alice_shared_value = alice_private_key.exchange(bob_public_key)
print("\nShared secret value:\n", hexlify(alice_shared_value))
alice_derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,  # Важливо не додавати рандомізацію для отримання однакового ключа з обох сторін.
    info=b"handshake data",
).derive(alice_shared_value)
print("\nDerived secret key:\n", hexlify(alice_derived_key))


Shared secret value:
 b'35719226307d933245cdb8c938287592c5195bffaf95d88d97fb26ed72f859bbe50840ef38498204c293bb6470e352b03dabaee11acdea73d68935ee92e0aa6192d3afa0d2ba719c147df6a375e1052cb9ffb88a5fd6d8a4956292ab47eaeb643ef5c52e0f4b9bde207b9a2b81d40729146718ce64c34d4c2a86ede86e7f7ecd10f6d2f6244462ce74350cf1d7254d49d19f8c1a44b37101ad646c42f55cd7cfda17feab10ff39f2d6471c41499ab279395e2d109017869d3aa2205141090ef38a675e2a8863a6b28c9f6a40ae009049dce5208166c951d194ba043a15212a7846fbd25511b378fb19c4b65c43bd439730abf07d6ae1429dbd9fa7ff8d43099b'

Derived secret key:
 b'0ac5cc0a823b7df6d62aeb803292326ed985844e3a13924659925a0247868336'


In [7]:
# Bob
bob_shared_value = bob_private_key.exchange(alice_public_key)
print("\nShared secret value:\n", hexlify(bob_shared_value))
bob_derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,  # Важливо не додавати рандомізацію для отримання однакового ключа з обох сторін.
    info=b"handshake data",
).derive(bob_shared_value)


Shared secret value:
 b'35719226307d933245cdb8c938287592c5195bffaf95d88d97fb26ed72f859bbe50840ef38498204c293bb6470e352b03dabaee11acdea73d68935ee92e0aa6192d3afa0d2ba719c147df6a375e1052cb9ffb88a5fd6d8a4956292ab47eaeb643ef5c52e0f4b9bde207b9a2b81d40729146718ce64c34d4c2a86ede86e7f7ecd10f6d2f6244462ce74350cf1d7254d49d19f8c1a44b37101ad646c42f55cd7cfda17feab10ff39f2d6471c41499ab279395e2d109017869d3aa2205141090ef38a675e2a8863a6b28c9f6a40ae009049dce5208166c951d194ba043a15212a7846fbd25511b378fb19c4b65c43bd439730abf07d6ae1429dbd9fa7ff8d43099b'
